In [1]:
import os
!pip install typesense

In [67]:
import typesense

client = typesense.Client({
    'nodes': [{
        'host': 'localhost', # For Typesense Cloud use xxx.a1.typesense.net
        'port': '8108',      # For Typesense Cloud use 443
        'protocol': 'http'   # For Typesense Cloud use https
    }],
    'api_key': '7abc8d8b-ce95-493a-ae2c-fb5d94f104cf',
    'connection_timeout_seconds': 500
})



In [104]:
import os
import dotenv
dotenv.load_dotenv("../.env")

notes_schema = {
    'name': 'notes',
    'fields': [
        {'name': 'note', 'type': 'string' },
        {'name': 'tags', 'type': 'string[]', 'facet': True, 'infix': True},
        {
            "name": "embedding",
            "type": "float[]",
            "embed": {
                "from": [
                    "note"
                ],
                "model_config": {
                    "model_name": "openai/text-embedding-3-small",
                    "api_key": os.getenv("OPENAI_NOTES_API_KEY")
                }
            }
        }

    ]
}

client.collections['notes'].delete()

client.collections.create(notes_schema)



{'created_at': 1732042711,
 'default_sorting_field': '',
 'enable_nested_fields': False,
 'fields': [{'facet': False,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'note',
   'optional': False,
   'sort': False,
   'stem': False,
   'store': True,
   'type': 'string'},
  {'facet': True,
   'index': True,
   'infix': True,
   'locale': '',
   'name': 'tags',
   'optional': False,
   'sort': False,
   'stem': False,
   'store': True,
   'type': 'string[]'},
  {'embed': {'from': ['note'],
    'model_config': {'api_key': 'sk-pr***************************************************************************************************************************************************************',
     'model_name': 'openai/text-embedding-3-small'}},
   'facet': False,
   'hnsw_params': {'M': 16, 'ef_construction': 200},
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'embedding',
   'num_dim': 1536,
   'optional': False,
   'sort': False,
   'stem': False,
   'stor

In [15]:
client.collections['notes'].documents.create({
    'note': 'This is a note',
    'tags': ['personal', 'work']
})

{'embedding': [0.027989011257886887,
  0.010720399208366871,
  -0.02161051519215107,
  0.004939445294439793,
  -0.027734437957406044,
  -0.033745214343070984,
  0.004119150806218386,
  0.021058937534689903,
  -0.01643417403101921,
  -7.077028567437083e-05,
  -0.014977444894611835,
  0.01100325956940651,
  -0.026023132726550102,
  -0.01018296554684639,
  0.05196140706539154,
  0.022402523085474968,
  -0.03136919066309929,
  -0.0295305997133255,
  -0.011964984238147736,
  0.0008335534948855639,
  0.018216192722320557,
  0.007361435331404209,
  0.040449000895023346,
  0.03261377289891243,
  0.015514878556132317,
  -0.03241577371954918,
  -0.006795715074986219,
  -0.026928285136818886,
  0.0036913249641656876,
  0.004769728984683752,
  0.04989653080701828,
  -0.04695478454232216,
  -0.00021325003763195127,
  -0.02432597242295742,
  -0.029558885842561722,
  0.026277707889676094,
  0.04432418569922447,
  -0.00985767599195242,
  -0.011618480086326599,
  -0.03351892530918121,
  -0.046049632132

In [16]:
search_parameters = {
    'q'         : 'is',
    'query_by'  : 'note'
}

client.collections['notes'].documents.search(search_parameters)



{'facet_counts': [],
 'found': 1,
 'hits': [{'document': {'embedding': [0.027989011257886887,
     0.010720399208366871,
     -0.02161051519215107,
     0.004939445294439793,
     -0.027734437957406044,
     -0.033745214343070984,
     0.004119150806218386,
     0.021058937534689903,
     -0.01643417403101921,
     -7.077028567437083e-05,
     -0.014977444894611835,
     0.01100325956940651,
     -0.026023132726550102,
     -0.01018296554684639,
     0.05196140706539154,
     0.022402523085474968,
     -0.03136919066309929,
     -0.0295305997133255,
     -0.011964984238147736,
     0.0008335534948855639,
     0.018216192722320557,
     0.007361435331404209,
     0.040449000895023346,
     0.03261377289891243,
     0.015514878556132317,
     -0.03241577371954918,
     -0.006795715074986219,
     -0.026928285136818886,
     0.0036913249641656876,
     0.004769728984683752,
     0.04989653080701828,
     -0.04695478454232216,
     -0.00021325003763195127,
     -0.02432597242295742,
     -

In [18]:
!pip install openai

In [55]:
from openai import OpenAI
oai = OpenAI(api_key=os.getenv("OPENAI_NOTES_API_KEY"))
oai.api_key = os.getenv("OPENAI_NOTES_API_KEY")

embeddings_model_used = "text-embedding-3-small"
def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return oai.embeddings.create(input = [text], model=model).data[0].embedding

#embedding = get_embedding("note", model='text-embedding-3-small')

In [46]:
def search(embedding: [float]):
    
    search_requests = {
        'searches': [
            {
                'collection': 'notes',
                'q' : '*',
                'vector_query': f'embedding:({embedding}, k:100)', # <=== Be sure to replace `embedding` with the name of the field that stores your embeddings.
                'exclude_fields': 'embedding' # <=== Don't return the raw floating point numbers in the vector field in the search API response, to save on network bandwidth.
            }
        ]
    }
    
    # Search parameters that are common to all searches go here
    common_search_params =  {}
    return client.multi_search.perform(search_requests, common_search_params)
    


In [31]:
from sleep_events import connect_to_firebase

firebase = connect_to_firebase()

In [73]:
import pandas as pd

docs = firebase.collection('notesExperimental').stream()

note_records = [{'id': doc.id, **doc.to_dict()} for doc in docs]

note_df = pd.DataFrame(note_records)

In [74]:
note_records[0]

{'id': '2024-05-03T16:48:08.446+01:00',
 'timestampWritten': DatetimeWithNanoseconds(2024, 5, 3, 15, 48, 8, 446000, tzinfo=datetime.timezone.utc),
 'timestampForUTC': DatetimeWithNanoseconds(2024, 5, 3, 15, 47, 17, 610000, tzinfo=datetime.timezone.utc),
 'timestampWrittenUTC': DatetimeWithNanoseconds(2024, 5, 3, 15, 48, 8, 446000, tzinfo=datetime.timezone.utc),
 'note': "Interesting ritalin thing.. It suppresses my inhibition enough to make me respond quicker, and that's really making me realise that I am a bit autistic. Maybe?",
 'tags': ['ritalin', 'inhibition', 'autistic'],
 'timestampFor': DatetimeWithNanoseconds(2024, 5, 3, 15, 47, 17, 610000, tzinfo=datetime.timezone.utc),
 'dayAndNightOf': '2024-05-03'}

In [35]:
note_df

,dayAndNightOf,timestampWrittenUTC,timestampForUTC,timestampWritten,tags,timestampFor,note,timestampUpdated,isPreviousDay
0,2024-05-03,2024-05-03 15:48:08.446000+00:00,2024-05-03 15:47:17.610000+00:00,2024-05-03 15:48:08.446000+00:00,"[ritalin, inhibition, autistic]",2024-05-03 15:47:17.610000+00:00,Interesting ritalin thing.. It suppresses my i...,NaT,NaN
1,2024-05-03,2024-05-03 21:16:32.280000+00:00,2024-05-03 21:14:39.301000+00:00,2024-05-03 21:16:32.280000+00:00,"[poor-settling-leads-to-poor-sleep, sleep, rit...",2024-05-03 21:14:39.301000+00:00,Last night stayed up on laptop until last minu...,NaT,NaN
2,2024-05-04,2024-05-04 09:04:52.842000+00:00,2024-05-04 09:04:52.841000+00:00,2024-05-04 09:04:52.842000+00:00,[diary],2024-05-04 09:04:52.841000+00:00,Going to Helen's today for Laura's 2nd 30th bi...,NaT,NaN
3,2024-05-05,2024-05-04 21:48:56.424000+00:00,2024-05-05 21:48:56+00:00,2024-05-04 21:48:56.424000+00:00,[],2024-05-05 21:48:56+00:00,Completely knackered but coming back from thea...,NaT,NaN
4,2024-05-05,2024-05-05 07:48:45.112000+00:00,2024-05-05 07:48:45+00:00,2024-05-05 07:48:45.112000+00:00,"[sleep:disturbance:cat, sleep:disturbance:sunl...",2024-05-05 07:48:45+00:00,I think I got disturbed during the night by th...,NaT,NaN
...,...,...,...,...,...,...,...,...,...
1888,2024-11-19,NaT,NaT,2024-11-19 14:04:39.333000+00:00,[],2024-11-19 14:04:39+00:00,Kinda tired currently,NaT,NaN
1889,2024-11-19,NaT,NaT,2024-11-19 14:51:09.646000+00:00,[],2024-11-19 14:51:09+00:00,Getting distracted. Hard to focus.,NaT,NaN
1890,2024-11-19,NaT,NaT,2024-11-19 15:13:13.562000+00:00,[],2024-11-19 15:13:13+00:00,"Been distracted for a while, struggling to get...",NaT,NaN
1891,2024-11-19,NaT,NaT,2024-11-19 15:58:45.424000+00:00,[],2024-11-19 15:58:45+00:00,Bah just can't get into work. May have to stop.,NaT,NaN


In [44]:
few_notes = note_records[0:1]
few_notes

[{'dayAndNightOf': '2024-05-03',
  'timestampWrittenUTC': DatetimeWithNanoseconds(2024, 5, 3, 15, 48, 8, 446000, tzinfo=datetime.timezone.utc),
  'timestampForUTC': DatetimeWithNanoseconds(2024, 5, 3, 15, 47, 17, 610000, tzinfo=datetime.timezone.utc),
  'timestampWritten': DatetimeWithNanoseconds(2024, 5, 3, 15, 48, 8, 446000, tzinfo=datetime.timezone.utc),
  'tags': ['ritalin', 'inhibition', 'autistic'],
  'timestampFor': DatetimeWithNanoseconds(2024, 5, 3, 15, 47, 17, 610000, tzinfo=datetime.timezone.utc),
  'note': "Interesting ritalin thing.. It suppresses my inhibition enough to make me respond quicker, and that's really making me realise that I am a bit autistic. Maybe?"}]

In [43]:
client.collections["notes"].documents.import_(few_notes)

TypeError: Object of type DatetimeWithNanoseconds is not JSON serializable

In [76]:
from datetime import datetime

# Helper function to convert DatetimeWithNanoseconds to ISO format
def serialize_datetime(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()
    return obj

# Convert datetime fields in few_notes
few_notes_serialized = []
for note in note_records:
    serialized_note = {key: serialize_datetime(value) if isinstance(value, datetime) else value for key, value in note.items()}
    few_notes_serialized.append(serialized_note)

# Import the serialized notes
# client.collections["notes"].documents.import_(few_notes_serialized)


In [77]:
few_notes_serialized[0]

{'id': '2024-05-03T16:48:08.446+01:00',
 'timestampWritten': '2024-05-03T15:48:08.446000+00:00',
 'timestampForUTC': '2024-05-03T15:47:17.610000+00:00',
 'timestampWrittenUTC': '2024-05-03T15:48:08.446000+00:00',
 'note': "Interesting ritalin thing.. It suppresses my inhibition enough to make me respond quicker, and that's really making me realise that I am a bit autistic. Maybe?",
 'tags': ['ritalin', 'inhibition', 'autistic'],
 'timestampFor': '2024-05-03T15:47:17.610000+00:00',
 'dayAndNightOf': '2024-05-03'}

In [105]:
client.collections["notes"].documents.import_(few_notes_serialized)


[{'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},


In [82]:
search(get_embedding("effects of ritalin + protein", model='text-embedding-3-small'))


{'results': [{'facet_counts': [],
   'found': 100,
   'hits': [{'document': {'dayAndNightOf': '2024-05-25',
      'id': '2024-05-25T11:16:19.864+01:00',
      'note': "1115: 1hr15 since ritalin but it's not doing it's normal thing.\nThis is the thing with ritalin. Sometimes it just doesn't work, or work the same way?  I took it from the packet in the living room and normally I get it from my bedroom stash. Had a protein breakfast. Have been feeling not-wonderful, not-awful today.",
      'tags': ['ritalin'],
      'timestampFor': '2024-05-25T10:16:19+00:00',
      'timestampWritten': '2024-05-25T10:16:19.864000+00:00'},
     'highlight': {},
     'highlights': [],
     'vector_distance': 0.3906896114349365},
    {'document': {'dayAndNightOf': '2024-09-15',
      'id': '2024-09-15T15:25:44.156+01:00',
      'note': 'Tired when ritalin runs out',
      'tags': [],
      'timestampFor': '2024-09-15T14:25:44+00:00',
      'timestampUpdated': None,
      'timestampWritten': '2024-09-15T14:2

In [54]:
!pip install tiktoken

   ---------------------------------------- 0.0/884.2 kB ? eta -:--:--
   ---------------------------------------- 884.2/884.2 kB 5.0 MB/s eta 0:00:00


In [61]:
import tiktoken

def estimate_token_count(text, embeddings_model_type):
    encoding = tiktoken.encoding_for_model(embeddings_model_type)
    tokens = encoding.encode(text)
    return len(tokens)

def calculate_total_cost(token_count, embeddings_model_type):
    return (token_count / 1_000_000) * 0.020

total_token_count = sum(estimate_token_count(doc['note'], embeddings_model_used) for doc in note_records)
total_cost = calculate_total_cost(total_token_count, embeddings_model_used)
print(f"Total estimated cost: ${total_cost:.4f} for {total_token_count} tokens using model {embeddings_model_used}")


Total estimated cost: $0.0011 for 55201 tokens using model text-embedding-3-small


In [123]:
# Perform a search query to retrieve all documents
search_parameters = {
    'q': '*',
    'include_fields': 'tags',
    # 'group_by': 'tags',
    'per_page': 5000  # Adjust this number based on the expected number of documents
}

response = client.collections['notes'].documents.search(search_parameters)
# response
# Extract tags from the search results
all_tags = set()
for document in response['hits']:
    tags = document['document'].get('tags', [])
    all_tags.update(tags)

print(all_tags)

ObjectUnprocessable: [Errno 422] Only upto 250 hits can be fetched per page.

In [127]:
all_tags = set()

def fetch_all_documents():
    all_documents = []
    page = 1
    per_page = 250  # Adjust this number based on the expected number of documents per page

    while True:
        print(f"Fetching page {page}...")
        search_parameters = {
            'q': '*',
            'include_fields': 'tags',
            'per_page': per_page,
            'page': page
        }

        response = client.collections['notes'].documents.search(search_parameters)
        hits = response['hits']

        if not hits:
            break

        #all_documents.extend(hits)
        for document in hits:
            tags = document['document'].get('tags', [])
            all_tags.update(tags)

        page += 1

    return all_documents

all_documents = fetch_all_documents()
print(f"Total documents fetched: {len(all_documents)}")
print(len(all_tags))

Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Fetching page 6...
Fetching page 7...
Fetching page 8...
Fetching page 9...
Total documents fetched: 0
257


In [128]:
all_tags

{'5htp',
 'adhd',
 'adhd:obsessive-research',
 'adhd:protein',
 'air-purifier',
 'apnea',
 'appetite:exercise',
 'appetite:sun',
 'autistic',
 'awake-early',
 'bath',
 'bedroom-heater',
 'blood-pressure',
 'breathing',
 'chamomile',
 'circadian-rhythm',
 'circadian-rhythm: shower',
 'circadian-rhythm:alarm',
 'circadian-rhythm:daylight-savings',
 'circadian-rhythm:energy',
 'circadian-rhythm:heart-rate:early-evening',
 'circadian-rhythm:lep',
 'circadian-rhythm:lep:late-night',
 'circadian-rhythm:lep:sleepiness',
 'circadian-rhythm:second-wind',
 'circadian-rhythm:summary',
 'cleaning',
 'co2',
 'concerta:27mg',
 'concerta:cleaning',
 'concerta:drinking',
 'concerta:emotional',
 'concerta:ending',
 'concerta:insomnia',
 'concerta:kicking-in',
 'concerta:protein',
 'concerta:settling',
 'concerta:skipped-breakfast',
 'concerta:social',
 'concerta:vitamin-c',
 'context',
 'context:fatigue',
 'data',
 'data:core',
 'data:food',
 'data:lux',
 'data:viatom',
 'data:yasa',
 'dehydration',
 '

In [168]:
search_parameters = {
    'q': '*',
    'filter_by': 'tags:[*insomnia*]',
    'infix': 'always',
    'per_page': 3
}

response = client.collections['notes'].documents.search(search_parameters)
for hit in response['hits']:
    print("Note: " + hit['document']['note'])
    print(f"Tags: {hit['document']['tags']}")

Note: Really struggling to get into XL now family has left. Not feeling very affected by rit.
Tags: ['ritalin']
Note: Rit is definitely applying now. As ever quite hard to work out with this brand when that happened.
Tags: ['ritalin']
Note: Tired and sleepy.
Had 5mg of my old rit and 5mg of new today. I'm not sure this new stuff actually works..
Tags: ['ritalin']
Note: It's up and down. Feel pretty good when rit is in, pretty tired when its not. First time with new brand of rit, 5mg, and it seems to be working as usual.
Tags: ['ritalin:5mg', 'ritalin']
Note: Feel conderta finally starting to kick in.
From the graph it is meant to work like this, and does mean I don't really feel medicated until nearly 3 hours in. Maybe ritalin is actually better?
Tags: ['concerta:kicking-in', 'ritalin']
Note: Have perked up a fair bit now, maybe being part of an active meetig and/or ritalin kicking in
Tags: ['ritalin']
Note: I'm not sure this 30mg dose is working for me.  Not only have I got a bunch of